In [ ]:
import gym
from gym import Env, spaces

import time
import numpy as np
import seaborn as sns
from plotly.offline import iplot
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import random
import networkx as nx
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerPatch,HandlerTuple
import matplotlib.patches as patches
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

import gymnasium as gym
from gymnasium import Env, spaces

from sklearn import preprocessing


In [2]:

num_drones = 4
limite_y = 80
limite_x = 100
alcance_comunicacao_jammer = 25
alcance_comunicacao_nos = 5000
B_Hz = 2.4e9
posicoes = np.array([[18, 52], [33, 20], [44, 15], [48, 25]])
posicao_jammer = np.array([40, 70])

In [3]:
# Definir a subgrade dentro da área maior
area_size = (100, 100)
subgrade_start = (20, 20)  # Ponto inicial da subgrade (x, y)
subgrade_size = (50, 40)   # Tamanho da subgrade (largura, altura)
resolution = 5             # Resolução da subgrade (distância entre pontos)

# Gerar os pontos discretizados na subgrade
x_points = np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + 1, resolution)
y_points = np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + 1, resolution)
grid_points = np.array(np.meshgrid(x_points, y_points)).T.reshape(-1, 2)


In [ ]:
# Função para calcular a distância entre duas posições
def distancia(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

# Função para encontrar vizinhos dentro do alcance de comunicação
def encontra_vizinhos(posicoes, alcance_comunicacao_nos):
    num_drones = posicoes.shape[0]
    vizinhos = {}
    for i in range(num_drones):
        vizinhos[i] = []
        for j in range(num_drones):
            if i != j and distancia(posicoes[i], posicoes[j]) <= alcance_comunicacao_nos:
                vizinhos[i].append(j)
    return vizinhos

# Função para verificar quais drones são afetados pelo jammer
def verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer):
    afetados_pelo_jammer = []
    for i in range(posicoes.shape[0]):
        if distancia(posicoes[i], posicao_jammer) <= alcance_comunicacao_jammer:
            afetados_pelo_jammer.append(i)
    return afetados_pelo_jammer

# Gera direções aleatórias para as antenas
direcoes_antena = np.random.uniform(0, 360, size=num_drones)
# direcoes_antena = [278, 102, 235, 312, 182, 242]
afetados_pelo_jammer=verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer)
# Comprimento das setas para indicar a direção das antenas
comprimento_seta = 5



# Visualização usando Matplotlib
def render_v2(posicoes, direcoes_antena, posicao_jammer, comprimento_seta=5, area_size=(100, 100)):
    afetados_pelo_jammer = verifica_jammer(posicoes, posicao_jammer, alcance_comunicacao_jammer)

    fig, ax = plt.subplots(figsize=(9, 4))

    # Desenha os drones no gráfico
    for i, pos in enumerate(posicoes):
        ax.scatter(pos[0], pos[1], color='blue', s=100, label='Drone' if i == 0 else "")
        ax.text(pos[0], pos[1] - 2, f'{i}', horizontalalignment='center', color='white', fontweight='bold')

    # Desenha as setas para a direção das antenas
    for i, direcao in enumerate(direcoes_antena):
        direcao_rad = np.radians(direcao)  # Converte para radianos
        dx = comprimento_seta * np.cos(direcao_rad)  # Deslocamento em x
        dy = comprimento_seta * np.sin(direcao_rad)  # Deslocamento em y
        
        ax.arrow(
            posicoes[i][0], posicoes[i][1], 
            dx, dy, 
            head_width=2, head_length=2, fc='b', ec='b'
        )  # Seta azul para indicar a direção da antena

    # Desenha o jammer
    ax.scatter(posicao_jammer[0], posicao_jammer[1], color='red', s=100, marker='o', label='Jammer')

    # Configurações finais do gráfico
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_xlim(0, area_size[0])
    ax.set_ylim(0, area_size[1])
    ax.set_yticks(np.arange(0, area_size[1]+1, 20))
    plt.title('Direções das Antenas UAV')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize='small')

    plt.show()

render_v2(posicoes, direcoes_antena, posicao_jammer, comprimento_seta)


In [ ]:
import numpy as np

# Função para calcular o ângulo entre dois nós em relação à direção da antena
def angulo_entre_nos(pos1, direcao_antena, pos2):
    # Vetor entre os dois nós
    delta_x = pos2[0] - pos1[0]
    delta_y = pos2[1] - pos1[1]
    
    # Direção do vetor em graus
    direcao_vetor_rad = np.arctan2(delta_y, delta_x)
    direcao_vetor_deg = np.degrees(direcao_vetor_rad)
    
    # Diferença entre a direção do vetor e a direção da antena
    angulo = direcao_vetor_deg - direcao_antena
    
    # Ajustar para o intervalo de 0 a 360 graus
    angulo = angulo % 360  # Usa módulo para garantir que o valor esteja entre 0 e 360
    angulo_arredondado = round(angulo)
    
    return angulo_arredondado

# Posições dos drones
# posicoes = np.array([[16, 30], [18, 48], [44, 15], [23, 73], [45, 73], [50, 50]])

# Direções aleatórias das antenas
# direcoes_antena = np.random.uniform(0, 360, size=posicoes.shape[0])

# Matriz para armazenar os ângulos
angulos_matriz = np.zeros((posicoes.shape[0], posicoes.shape[0]))

# Calcular os ângulos para todas as combinações de drones
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            angulos_matriz[i, j] = angulo_entre_nos(posicoes[i], direcoes_antena[i], posicoes[j])

# Apresentar os ângulos entre drones sem valores negativos
for i in range(posicoes.shape[0]):
    print(f"Drones que o drone {i} 'vê' com um ângulo:")
    for j in range(posicoes.shape[0]):
        if i != j:
            angulo_arredondado = int(round(angulos_matriz[i, j]))  # Arredondar para unidade
            print(f"  Com o drone {j}: {angulo_arredondado} graus")

In [6]:
Ptx_dBm = 20 
f = 2.4e9 
B_Hz= 2.4e9
d0=1
gamma=2
sigma = 0
c=3e8
lambda_m=c/f
L0=30
potencia_jammer_dBm =100

# Ler o arquivo de ganhos para criar uma tabela de busca
ganhos_df = pd.read_csv('ganhos.csv')

In [ ]:
angulos = np.deg2rad(ganhos_df['angulo'])  # Converter graus para radianos
ganhos = ganhos_df['ganho']

# Criar um gráfico polar
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(angulos, ganhos, linestyle='-', linewidth=1, label='Ganho da Antena')

# Ajustar o layout
ax.set_theta_zero_location('N')  # Configurar a direção Norte como o topo do gráfico
ax.set_theta_direction(-1)  # Configurar a direção dos ângulos para sentido horário
ax.set_rmax(max(ganhos) + 1)  # Ajustar o raio máximo para melhor visualização
ax.set_rticks(np.linspace(min(ganhos), max(ganhos), num=5))  # Definir os ticks para o raio
ax.set_rlabel_position(45)  # Posicionar os labels do raio

# Configurar os ticks para o ângulo
ax.set_xticks(np.deg2rad(np.arange(0, 360, 30)))  # Definir marcas de ângulo a cada 30 graus

# Adicionar legenda e título
ax.legend()
ax.set_title('Ganho da Antena por Ângulo', va='bottom')

# Mostrar o gráfico
plt.show()

In [ ]:
def busca_ganhos( angulo):
    # Arredondar para o ângulo inteiro mais próximo
    angulo_ajustado = int(round(angulo))
    
    if angulo_ajustado==360:
        angulo_ajustado=0
    
    # Encontrar o ganho correspondente ao ângulo ajustado
    ganho = ganhos_df.loc[ganhos_df['angulo'] == angulo_ajustado, 'ganho'].iloc[0]
    
    return ganho



# Função para calcular os ganhos de transmissão e recepção
def calcula_ganhos(posicoes, direcoes_antena, indice_transmissor, indice_receptor, ganhos_df):
    # Obter posições e direções
    pos1 = posicoes[indice_transmissor]
    pos2 = posicoes[indice_receptor]
    direcao_antena1 = direcoes_antena[indice_transmissor]
    direcao_antena2 = direcoes_antena[indice_receptor]
    
    # Calcular o ângulo entre os dois nós em relação à direção da antena
    angulo_transmissao = angulo_entre_nos(pos1, direcao_antena1, pos2)
    angulo_rececao = angulo_entre_nos(pos2, direcao_antena2, pos1)
    # print('Angulo receção:',angulo_rececao)
    # print('Angulo tranmissão:',angulo_transmissao)

    # Obter os ganhos para transmissão e recepção
    ganho_transmissao = busca_ganhos( angulo_transmissao)
    ganho_rececao = busca_ganhos( angulo_rececao)
    
    return ganho_transmissao, ganho_rececao

# ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, 0, 1, ganhos_df)



def calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d,f):
    """
    Calcula a potência do sinal recebido usando a equação de Friis.
    """
    # L = (4 * np.pi * d / lambda_m)**2
    # Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - 10 * np.log10(L)
    # return Prx_dBm
    # Calcular a perda de percurso
    if d > 0:
        L = L0 + 10 * gamma * np.log10(d / d0) + np.random.normal(0, sigma)
    else:
        L = L0  # Se a distância for zero, assume-se a perda na distância de referência

    # Calcular a potência do sinal recebido
    Prx_dBm = Ptx_dBm + ganho_transmissao + ganho_rececao - L

    return Prx_dBm

def calcula_potencia_jammer(pos_drone, direcao_antena_drone, pos_jammer, potencia_jammer_dBm, ganhos_df):
    # Calcular a distância entre o drone e o jammer
    d = distancia(pos_drone, pos_jammer)
    
    # Calcular o ângulo entre o drone e o jammer
    angulo_entre_drone_e_jammer = angulo_entre_nos(pos_drone, direcao_antena_drone, pos_jammer)
    
    # Obter o ganho do drone em direção ao jammer
    ganho_drone_para_jammer = busca_ganhos( angulo_entre_drone_e_jammer)
    
    # Calcular a potência do ruído recebido usando a equação de Friis
    L = (4 * np.pi * d / lambda_m) ** 2  # Perda de propagação
    potencia_jammer_recebida_dBm = potencia_jammer_dBm + ganho_drone_para_jammer - 10 * np.log10(L)
    
    return potencia_jammer_recebida_dBm

def calcula_capacidade(potencia_sinal_dBm, potencia_ruido_dBm):
    """
    Calcula a capacidade do canal em bits por segundo usando a fórmula de Shannon-Hartley.
    """
    # Converter potência em dBm para Watts
    potencia_sinal_W = 10 ** (potencia_sinal_dBm / 10) / 1000  # Potência do sinal em Watts
    potencia_ruido_W = 10 ** (potencia_ruido_dBm / 10) / 1000  # Potência do ruído em Watts
    
    # Calcular a razão sinal-ruído (SNR)
    snr = potencia_sinal_W / potencia_ruido_W
    
    # Calcular a capacidade do canal em bits por segundo (bps)
    capacidade = B_Hz * np.log2(1 + snr)
    
    return capacidade/1e3   

# -------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------
for i in range(posicoes.shape[0]):
    for j in range(posicoes.shape[0]):
        if i != j:
            # Calcular a distância entre os dois drones
            d = distancia(posicoes[i], posicoes[j])
            
            # Calcular os ganhos de transmissão e recepção
            ganho_transmissao, ganho_rececao = calcula_ganhos(posicoes, direcoes_antena, i, j, ganhos_df)
            # print(posicoes)
            print('Ganhos Rx,Tx:',ganho_transmissao, ganho_rececao)
            # Calcular a potência do sinal recebido usando a equação de Friis
            potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
            potencia_ruido = calcula_potencia_jammer(posicoes[j], direcoes_antena[j], posicao_jammer, potencia_jammer_dBm, ganhos_df)
            capacidade_canal=calcula_capacidade(potencia_recebida,potencia_ruido)
            # Imprimir a combinação de nós e a potência do sinal recebido
            print(f"  Potencia recebida do drone {i} para o drone {j}: {potencia_recebida:.2f} dBm")
            print(f"  Ruido recebido no drone {j} e do jammer: {potencia_ruido:.2f} dBm")
            print(f"  Capacidade do canal {i} para {j}: {capacidade_canal:.2f} bps")



In [9]:
seed=0

class UAVCommunicationEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, num_uavs=num_drones, area_size=(150, 90)):
        super().__init__()
        self.num_uavs = num_uavs
        self.area_size = area_size
        self.posicoes = posicoes.copy()
        self.posicao_jammer = posicao_jammer
        self.ultimo_info = {}

        # Define o espaço de ação: direções das antenas + deslocamentos x e y
        self.action_space = spaces.Box(low=np.array([0]*num_uavs + [-40]*num_uavs + [-40]*num_uavs), 
                                       high=np.array([360]*num_uavs + [40]*num_uavs + [40]*num_uavs), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=360, shape=(num_uavs,), dtype=np.float32)

        self.direcoes_antena = np.random.uniform(0, 360, size=num_uavs)
        self.capacidades = []  # Inicializa a lista de capacidades para normalização
        self.subgrade_size = subgrade_size
        self.subgrade_start = subgrade_start
        self.ganhos_df = ganhos_df
        self.subgrade_move_step = 10  # Passo de movimento da subgrade para a direita
        
        self.initialize_grid_points()

    def move_subgrade_right(self):
        """Move a subgrade para a direita em 10 unidades."""
        # Calcular a nova posição de início da subgrade
        new_start_x = self.subgrade_start[0] + self.subgrade_move_step
        
        # # Verificar se a subgrade ultrapassa os limites da área e ajustar se necessário
        # if new_start_x + self.subgrade_size[0] <= self.area_size[0]:
        #     self.subgrade_start = (new_start_x, self.subgrade_start[1])
        # else:
        #     self.subgrade_start = (self.area_size[0] - self.subgrade_size[0], self.subgrade_start[1])

    def step(self, action):
        # Mover a subgrade para a direita antes de executar a ação
        # self.move_subgrade_right()

        direcoes_antena = action[:self.num_uavs]
        movimentos_x = action[self.num_uavs:2*self.num_uavs]
        movimentos_y = action[2*self.num_uavs:]

        self.direcoes_antena = np.array(direcoes_antena)
        
        # for i in range(self.num_uavs):
        #     nova_pos_x = self.posicoes[i][0] + movimentos_x[i]
        #     nova_pos_y = self.posicoes[i][1] + movimentos_y[i]
        #     nova_pos_x = min(max(nova_pos_x, self.subgrade_start[0]), self.subgrade_start[0] + self.subgrade_size[0])
        #     nova_pos_y = min(max(nova_pos_y, self.subgrade_start[1]), self.subgrade_start[1] + self.subgrade_size[1])
        #     self.posicoes[i] = np.array([nova_pos_x, nova_pos_y])

        capacidades_por_link = []

        for i in range(self.posicoes.shape[0]):
            for j in range(self.posicoes.shape[0]):
                if i != j:
                    d = distancia(self.posicoes[i], self.posicoes[j])
                    ganho_transmissao, ganho_rececao = calcula_ganhos(self.posicoes, self.direcoes_antena, i, j, self.ganhos_df)
                    potencia_recebida = calcula_potencia_recebida(Ptx_dBm, ganho_transmissao, ganho_rececao, d, lambda_m)
                    potencia_ruido = calcula_potencia_jammer(self.posicoes[j], self.direcoes_antena[j], self.posicao_jammer, potencia_jammer_dBm, self.ganhos_df)
                    capacidade_canal = calcula_capacidade(potencia_recebida, potencia_ruido)
                    capacidades_por_link.append(capacidade_canal)

        capacidade_media = np.mean(capacidades_por_link)
        capacidade_minima = np.min(capacidades_por_link)

        self.capacidades.extend(capacidades_por_link)
        if len(self.capacidades) > 1000:
            self.capacidades = self.capacidades[-1000:]

        G = nx.Graph()
        num_drones = self.posicoes.shape[0]
        capacidade_matriz = np.zeros((self.num_uavs, self.num_uavs))
        link_index = 0

        for i in range(self.num_uavs):
            for j in range(self.num_uavs):
                if i != j:
                    capacidade_matriz[i, j] = capacidades_por_link[link_index]
                    link_index += 1

        for i in range(num_drones):
            for j in range(i + 1, num_drones):
                capacidade = capacidade_matriz[i, j]
                G.add_edge(i, j, weight=1.0 / capacidade)

        bottlenecks = {}
        for i in range(num_drones):
            for j in range(num_drones):
                if i != j:
                    try:
                        caminho = nx.dijkstra_path(G, source=i, target=j, weight='weight')
                        menor_capacidade = float('inf')
                        for k in range(len(caminho) - 1):
                            cap_atual = capacidade_matriz[caminho[k], caminho[k+1]]
                            if cap_atual < menor_capacidade:
                                menor_capacidade = cap_atual
                        bottlenecks[(i, j)] = (caminho, menor_capacidade)
                    except nx.NetworkXNoPath:
                        print(f"Não há caminho do nó {i} para o nó {j}")

        capacidade_bottleneck_media = np.mean([bottleneck for caminho, bottleneck in bottlenecks.values()])
        capacidade_bottleneck_minima = np.min([bottleneck for caminho, bottleneck in bottlenecks.values()])

        distancia_jammer = np.mean([distancia(pos, self.posicao_jammer) for pos in self.posicoes])
        penalidade_distancia_jammer = -(distancia_jammer ** 2)  # Ajustado para escala apropriada

        distancias_entre_drones = [distancia(self.posicoes[i], self.posicoes[j]) for i in range(num_drones) for j in range(i+1, num_drones)]
        distancia_minima_entre_drones = np.min(distancias_entre_drones)
        penalidade_proximidade_drones = (max(0, (30 - distancia_minima_entre_drones) ** 2))  # Ajustado para escala apropriada

        # Ajuste da função de recompensa para refletir as penalidades não normalizadas
        recompensa = capacidade_media * capacidade_bottleneck_minima - (0 * penalidade_distancia_jammer + 0 * penalidade_proximidade_drones)

        capacidades_por_link = [round(capacidade, 3) for capacidade in capacidades_por_link]
        capacidade_media = round(capacidade_media, 3)

        info = {
            'Recompensa': recompensa,
            'Capacidade média [Kbps]': capacidade_media,
            'Capacidades_por_link [Kbps]': capacidades_por_link,
            'Matriz de Capacidades [Kbps]': capacidade_matriz,
            'Capacidade mínima [Kbps]': capacidade_minima,
            'Capacidade bottleneck média': capacidade_bottleneck_media,
            'Capacidade bottleneck mínima': capacidade_bottleneck_minima,
            'Penalidade distância ao jammer': penalidade_distancia_jammer,
            'Penalidade proximidade entre drones': penalidade_proximidade_drones
        }

        done = False
        return np.array(self.direcoes_antena), recompensa, done, info

    def render(self, mode='human'):
        sns.set(style="whitegrid")
        fig, ax = plt.subplots(figsize=(12, 7))

        subgrade_start = self.subgrade_start
        subgrade_size = self.subgrade_size
        resolution = 5

        # Desenhar subgrade com cores mais intensas e linhas finas
        rect = plt.Rectangle(subgrade_start, subgrade_size[0], subgrade_size[1], linewidth=1.5, edgecolor='dodgerblue', facecolor='none')
        ax.add_patch(rect)

        for x in np.arange(subgrade_start[0], subgrade_start[0] + subgrade_size[0] + resolution, resolution):
            ax.axvline(x, color='dodgerblue', linestyle='--', linewidth=0.7, alpha=0.8)
        for y in np.arange(subgrade_start[1], subgrade_start[1] + subgrade_size[1] + resolution, resolution):
            ax.axhline(y, color='dodgerblue', linestyle='--', linewidth=0.7, alpha=0.8)

        # Desenhar os drones com cores distintas
        for i, pos in enumerate(self.posicoes):
            ax.scatter(pos[0], pos[1], color='dimgray', s=200, edgecolor='white', linewidth=1, label='UAV' if i == 0 else "")
            ax.text(
                pos[0], pos[1] + 0.1,  # Ajuste 0.5 para centralizar o texto verticalmente
                f'{i}', 
                horizontalalignment='center', 
                verticalalignment='center', 
                color='white', fontweight='bold'
            )

        # Desenhar setas para indicar a direção de cada drone
        arrow_length = 3  # Comprimento da seta
        for i, (x, y) in enumerate(self.posicoes):
            direcao = np.deg2rad(self.direcoes_antena[i])  # Converte para radianos
            dx = arrow_length * np.cos(direcao)  # Comprimento da seta no eixo X
            dy = arrow_length * np.sin(direcao)  # Comprimento da seta no eixo Y
            ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=1, color='dimgray', width=0.003)

        # Desenhar o diagrama de radiação usando gradientes
        angulos = np.deg2rad(self.ganhos_df['angulo'])
        ganhos = self.ganhos_df['ganho']
        ganhos_normalizados = (ganhos - ganhos.min()) / (ganhos.max() - ganhos.min())
        escala = 6

        for i, (x, y) in enumerate(self.posicoes):
            direcao = self.direcoes_antena[i]
            angulos_rotacionados = angulos + np.deg2rad(direcao)
            contorno_x = x + ganhos_normalizados * np.cos(angulos_rotacionados) * escala
            contorno_y = y + ganhos_normalizados * np.sin(angulos_rotacionados) * escala
            ax.plot(contorno_x, contorno_y, color='dodgerblue', alpha=0.7, linestyle='-', linewidth=1.2)

        # Desenhar o jammer com um ícone diferente
        ax.scatter(self.posicao_jammer[0], self.posicao_jammer[1], color='red', s=150, marker='X', edgecolor='white', linewidth=1, label='Jammer')

        # Melhorar a legenda com um design organizado e mais legível
        handles = [
            mlines.Line2D([], [], color='dimgray', marker='o', linestyle='None', markersize=10, label='UAV'),
            mlines.Line2D([], [], color='red', marker='X', linestyle='None', markersize=10, label='Jammer')
        ]
        
        for i, pos in enumerate(self.posicoes):
            handles.append(
                mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                            label=f'UAV {i}: Pos ({pos[0]}, {pos[1]}), Dir {self.direcoes_antena[i]:.1f}°')
            )

        # Informações adicionais na legenda
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Capacidade Média [bps]: {cap_media:.2f}')
        )
        handles.append(
            mlines.Line2D([], [], color='none', marker='', linestyle='None', markersize=0,
                        label=f'Bottleneck [bps]: {bottleneck:.2f}')
        )
        
        # Configurar a legenda de forma elegante fora da área de plotagem
        ax.legend(
            loc='upper center', 
            bbox_to_anchor=(0.5, -0.08),  # Ajuste a altura para -0.2
            # fontsize='small', 
            ncol=3,  # Aumenta o número de colunas para que a legenda ocupe mais espaço horizontal
            handles=handles, 
            frameon=True,  # Adicionar borda à legenda para uma aparência mais organizada
            framealpha=0.8,  # Ajusta a transparência da borda da legenda
            fancybox=True,  # Borda arredondada na legenda
            borderpad=1,  # Aumenta o espaçamento interno da legenda
            columnspacing=2.0,  # Aumenta o espaçamento entre as colunas da legenda
            handletextpad=1.5  # Aumenta o espaçamento entre os marcadores e o texto
        )
        
        # Configurar eixos e título
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_xlim(0, self.area_size[0])
        ax.set_ylim(0, self.area_size[1])
        ax.set_yticks(np.arange(0, self.area_size[1] + 1, 20))
        
        plt.title(title, fontsize=14)
        
        # Ajustar o layout
        plt.tight_layout()

        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.15)  # Adjust these values as needed

        plt.show()

    def reset(self,seed=None):
        if seed is not None:
            np.random.seed(seed)
        self.direcoes_antena = np.random.uniform(0, 360, size=self.num_uavs)
        self.posicoes = posicoes.copy()
        obs = np.array(self.direcoes_antena)
        return obs, {}
    
    # ----------------------------MOVIMENTO DOS DRONES----------------------------------------
    def move_drones(self, delta_time):
        """Move os drones ao longo do eixo X a uma velocidade constante."""
        velocidade = 2  # 5 metros por segundo
        deslocamento = velocidade * delta_time
        for i in range(self.num_uavs):
            nova_pos_x = self.posicoes[i][0] + deslocamento
            # Garantir que o drone não ultrapasse os limites da área
            # nova_pos_x = min(nova_pos_x, self.area_size[0])
            self.posicoes[i][0] = nova_pos_x
    
    def initialize_grid_points(self):
        """Initialize grid points within the current subgrid."""
        resolution = 5  # Assume que a resolução é 5, ajuste conforme necessário
        x_points = np.arange(self.subgrade_start[0], self.subgrade_start[0] + self.subgrade_size[0] + 1, resolution)
        y_points = np.arange(self.subgrade_start[1], self.subgrade_start[1] + self.subgrade_size[1] + 1, resolution)
        self.grid_points = np.array(np.meshgrid(x_points, y_points)).T.reshape(-1, 2)
        
        
            
    # ----------------------------MOVIMENTO DA GRELHA----------------------------------------
    def move_subgrade(self, delta_time):
        """Move a subgrade para a direita em uma taxa constante."""
        velocidade_subgrade = 2  # 5 metros por segundo
        deslocamento = velocidade_subgrade * delta_time

        # Calcular a nova posição de início da subgrade
        new_start_x = self.subgrade_start[0] + deslocamento

        # Garantir que a subgrade permaneça dentro dos limites da área e tenha pontos discretos
        max_x_limit = self.area_size[0] - self.subgrade_size[0]  # Limite máximo de X para subgrade
        new_start_x = min(max(new_start_x, 0), max_x_limit)  # Limitar o movimento para estar dentro dos limites

        self.subgrade_start = (new_start_x, self.subgrade_start[1])

        # Gerar os pontos discretizados na subgrade
        self.initialize_grid_points()

In [ ]:
class GeneticAlgorithm:
    def __init__(self, env, grid_points, population_size=30, generations=30, mutation_rate=0.2, crossover_rate=0.9):
        self.env = env
        self.grid_points = grid_points
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate

        # Initialize subgrid discrete points based on current subgrid position
        self.update_discrete_grid_points()

    def update_discrete_grid_points(self):
        """
        Updates the discrete grid points available within the subgrid boundaries based on the current position of the subgrid.
        """
        self.subgrid_x_min = self.env.subgrade_start[0]
        self.subgrid_x_max = self.env.subgrade_start[0] + self.env.subgrade_size[0]
        self.subgrid_y_min = self.env.subgrade_start[1]
        self.subgrid_y_max = self.env.subgrade_start[1] + self.env.subgrade_size[1]

        # Filter grid points to be within the current subgrid bounds
        self.current_subgrid_points = [
            point for point in self.env.grid_points  # Ensure it uses env.grid_points
            if self.subgrid_x_min <= point[0] <= self.subgrid_x_max and
            self.subgrid_y_min <= point[1] <= self.subgrid_y_max
        ]

    def random_position_chromosome(self):
        # Ensure we use the updated discrete grid points
        self.update_discrete_grid_points()

        # Verifica se existem pontos disponíveis suficientes para todos os drones
        if len(self.current_subgrid_points) < self.env.num_uavs:
            raise ValueError("Não há pontos discretos suficientes na subgrade para todos os UAVs.")

        # Escolha posições únicas aleatórias a partir dos pontos disponíveis
        unique_positions = random.sample(self.current_subgrid_points, self.env.num_uavs)
        
        return np.array(unique_positions).flatten()

    def random_direction_chromosome(self):
        return np.random.uniform(0, 360, size=self.env.num_uavs)

    def evaluate_position_fitness(self, chromosome):
        positions = chromosome.reshape(self.env.num_uavs, 2)
        
        # Preserva o valor de X atual e apenas altera Y
        for i in range(self.env.num_uavs):
            positions[i, 0] = self.env.posicoes[i][0]  # Mantém o valor X original

        self.env.posicoes = positions
        directions = np.zeros(self.env.num_uavs)  # Direções fixas ou predefinidas
        obs, reward, done, info = self.env.step(np.concatenate([directions, np.zeros(self.env.num_uavs), np.zeros(self.env.num_uavs)]))
        return reward, info

    def evaluate_direction_fitness(self, directions):
        # Atualizar as direções das antenas
        self.env.direcoes_antena = directions
        obs, reward, done, info = self.env.step(np.concatenate([directions, np.zeros(self.env.num_uavs), np.zeros(self.env.num_uavs)]))
        return reward, info

    def select(self, population, fitness_scores):
        tournament_size = 5
        selected = []
        for _ in range(self.population_size):
            contenders = random.sample(list(enumerate(fitness_scores)), tournament_size)
            winner_index = max(contenders, key=lambda item: item[1][0])[0]
            selected.append(population[winner_index])
        return selected

    def crossover_positions(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, len(parent1) // 2 - 1)
            child1_positions = np.vstack((parent1[:point*2].reshape(-1, 2), parent2[point*2:].reshape(-1, 2)))
            child2_positions = np.vstack((parent2[:point*2].reshape(-1, 2), parent1[point*2:].reshape(-1, 2)))

            # Ensure the children have unique positions
            child1_positions = self.ensure_unique_positions(child1_positions.flatten())
            child2_positions = self.ensure_unique_positions(child2_positions.flatten())

            return child1_positions, child2_positions
        else:
            return parent1, parent2
        
    def crossover_directions(self, parent1, parent2):
        if random.random() < self.crossover_rate:
            point = random.randint(1, len(parent1) - 1)
            child1 = np.concatenate([parent1[:point], parent2[point:]])
            child2 = np.concatenate([parent2[:point], parent1[point:]])
            return child1, child2
        else:
            return parent1, parent2

    def mutate_positions(self, chromosome):
        self.update_discrete_grid_points()
        positions = chromosome.reshape(self.env.num_uavs, 2)
        used_positions = set(tuple(pos) for pos in positions)

        for i in range(self.env.num_uavs):
            if random.random() < self.mutation_rate:
                new_pos_index = np.random.choice(len(self.current_subgrid_points))
                new_position = tuple(self.current_subgrid_points[new_pos_index])

                # Ensure the new position is unique
                while new_position in used_positions:
                    new_pos_index = np.random.choice(len(self.current_subgrid_points))
                    new_position = tuple(self.current_subgrid_points[new_pos_index])

                positions[i] = new_position
                used_positions.add(new_position)

        
        posicoes_mutadas=self.ensure_unique_positions(positions.flatten())
        # print('mutate_positions',posicoes_mutadas)
        # Ensure mutated positions are unique within the grid points
        return posicoes_mutadas

    def mutate_directions(self, directions):
        for i in range(len(directions)):
            if random.random() < self.mutation_rate:
                mutation = random.uniform(-90, 90)
                directions[i] = (directions[i] + mutation) % 360
        return directions

    def ensure_unique_positions(self, chromosome):
        positions = chromosome.reshape(self.env.num_uavs, 2)
        unique_positions = []
        used_positions = set()

        # Convertemos os pontos discretos da subgrade para um conjunto de tuplas para comparação fácil
        subgrid_points_set = set(map(tuple, self.current_subgrid_points))

        for pos in positions:
            pos_tuple = tuple(pos)
            # Ensure each position is unique and within the available grid points
            while pos_tuple in used_positions or pos_tuple not in subgrid_points_set:
                new_pos_index = np.random.choice(len(self.current_subgrid_points))
                pos = self.current_subgrid_points[new_pos_index]
                pos_tuple = tuple(pos)
            unique_positions.append(pos)
            used_positions.add(pos_tuple)

        return np.array(unique_positions).flatten()
    
    
    
    
    def has_duplicate_positions(self, chromosome):
        """
        Verifica se há posições duplicadas em um cromossomo.
        """
        positions = chromosome.reshape(self.env.num_uavs, 2)
        positions_set = set(map(tuple, positions))
        return len(positions_set) != len(positions)
    
    

    def run_position_optimization(self):
        best_fitness_ever = -np.inf
        best_positions_ever = None
        best_info_ever = None

        # Initialize population with random positions within the subgrid boundaries
        population = [self.random_position_chromosome() for _ in range(self.population_size)]

        for generation in range(self.generations):
            # Evaluate fitness of each chromosome (set of positions)
            fitness_scores = [self.evaluate_position_fitness(chrom) for chrom in population]
            best_fitness_in_gen, best_info_in_gen = max(fitness_scores, key=lambda x: x[0])

            # Update the best overall positions if the current generation has a better solution
            if best_fitness_in_gen > best_fitness_ever:
                best_fitness_ever = best_fitness_in_gen
                best_positions_ever = population[fitness_scores.index((best_fitness_in_gen, best_info_in_gen))]
                best_info_ever = best_info_in_gen

            # Select parents for the next generation
            selected = self.select(population, fitness_scores)
            new_population = []

            # Generate new population through crossover and mutation
            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(selected, 2)
                child1, child2 = self.crossover_positions(parent1, parent2)

                # Ensure mutated children have unique positions
                mutated_child1 = self.mutate_positions(child1)
                mutated_child2 = self.mutate_positions(child2)
                
                # Ensure children have unique positions within themselves
                mutated_child1 = self.ensure_unique_positions(mutated_child1)
                mutated_child2 = self.ensure_unique_positions(mutated_child2)

                # Add only if no duplicates exist
                if not self.has_duplicate_positions(mutated_child1):
                    new_population.append(mutated_child1)
                if len(new_population) < self.population_size and not self.has_duplicate_positions(mutated_child2):
                    new_population.append(mutated_child2)

            # Ensure new population is within subgrid bounds and has no duplicates
            population = [self.ensure_positions_within_subgrid(chrom) for chrom in new_population[:self.population_size]]
            
            # Debugging: Print current generation's positions to identify duplicates
            # for i, chrom in enumerate(population):
                # print(f"Generation {generation}, Chromosome {i}: {chrom.reshape(self.env.num_uavs, 2)}")

            print(f"Generation {generation}: Best Fitness = {best_fitness_in_gen:.2f}")
            # print('run_position_otimization', best_positions_ever)

        return best_positions_ever, best_info_ever

    def ensure_positions_within_subgrid(self, chromosome):
        """
        Ensure that all positions in the chromosome are within the subgrid boundaries.
        """
        # Ensure we use the updated discrete grid points
        self.update_discrete_grid_points()
        positions = chromosome.reshape(self.env.num_uavs, 2)
        for i in range(self.env.num_uavs):
            positions[i, 0] = np.clip(positions[i, 0], self.subgrid_x_min, self.subgrid_x_max)
            positions[i, 1] = np.clip(positions[i, 1], self.subgrid_y_min, self.subgrid_y_max)
        return positions.flatten()

    def run_direction_optimization(self, best_positions):
        best_fitness_ever = -np.inf
        best_directions_ever = None
        best_info_ever = None

        population = [self.random_direction_chromosome() for _ in range(self.population_size)]

        for generation in range(self.generations):
            fitness_scores = [self.evaluate_direction_fitness(chrom) for chrom in population]
            best_fitness_in_gen, best_info_in_gen = max(fitness_scores, key=lambda x: x[0])

            if best_fitness_in_gen > best_fitness_ever:
                best_fitness_ever = best_fitness_in_gen
                best_directions_ever = population[fitness_scores.index((best_fitness_in_gen, best_info_in_gen))]
                best_info_ever = best_info_in_gen

            selected = self.select(population, fitness_scores)
            new_population = []

            while len(new_population) < self.population_size:
                parent1, parent2 = random.sample(selected, 2)
                child1, child2 = self.crossover_directions(parent1, parent2)
                new_population.extend([self.mutate_directions(child1), self.mutate_directions(child2)])

            population = new_population[:self.population_size]
            print(f"Generation {generation}: Best Fitness = {best_fitness_in_gen:.2f}")

        # Add the missing return statement
        return best_directions_ever, best_info_ever
    
    # Funções Extra para o Movimento
    def run_position_optimization_with_movement(self, total_duration, update_interval):
        """
        Executa a otimização de posições com movimento dos drones.
        O algoritmo genético é executado a cada `update_interval` segundos.
        """
        # Move os drones após a execução do GA
        self.env.move_drones(update_interval)
        self.env.move_subgrade(update_interval)

        # Update discrete grid points after moving the subgrid
        self.update_discrete_grid_points()

        # Ensure there are available points in the new subgrid position
        if len(self.current_subgrid_points) == 0:
            raise ValueError("Subgrid moved to an area with no available discrete points.")

        # Executa a otimização de posições
        best_positions, best_info = self.run_position_optimization()
        print(f"Melhores Posições na Iteração:\n{best_positions}")

        return best_positions, best_info
    
    
    def adjust_duplicate_positions(self, chromosome):
        """
        Ajusta posições duplicadas no cromossomo para a posição livre mais próxima.
        """
        positions = chromosome.reshape(self.env.num_uavs, 2)
        unique_positions = set()
        adjusted_positions = []

        for pos in positions:
            pos_tuple = tuple(pos)
            if pos_tuple in unique_positions:
                # Encontrar a posição livre mais próxima
                closest_position = self.find_closest_free_position(pos_tuple, unique_positions)
                adjusted_positions.append(np.array(closest_position))
                unique_positions.add(closest_position)
            else:
                adjusted_positions.append(pos)
                unique_positions.add(pos_tuple)

        return np.array(adjusted_positions).flatten()

    def find_closest_free_position(self, position, used_positions):
        """
        Encontra a posição livre mais próxima de uma dada posição que não está em used_positions.
        """
        min_distance = float('inf')
        closest_position = None

        for point in self.current_subgrid_points:
            if tuple(point) not in used_positions:
                distance = np.linalg.norm(np.array(position) - np.array(point))
                if distance < min_distance:
                    min_distance = distance
                    closest_position = tuple(point)

        return closest_position
    
    
    
    
    
    
    
    
    

# Exemplo de uso do algoritmo dividido em duas fases
env = UAVCommunicationEnv()
env.posicao_jammer=[65,40]
# Criar uma instância do Algoritmo Genético
ga = GeneticAlgorithm(env, grid_points)

# Configuração do movimento e otimização
total_duration = 60  # Total de 60 segundos
update_interval = 7.5  # Executa o GA a cada 15 segundos


env.posicoes = np.array([[35, 50], [25, 25], [45, 35], [45, 60]])
env.direcoes_antena=[45.2, 67.5, 102.5, 210]
title= 'Direções e Posições Estabelecidas - Algoritmo SAC'
env.render()
current_time = 0
while current_time < total_duration:
    start_time = time.time()
    
    print('---------------------------Otimização com Movimento------------------------------')
    best_positions, best_info = ga.run_position_optimization_with_movement(total_duration, update_interval)

    # Ajustar posições duplicadas para a posição livre mais próxima
    best_positions = ga.adjust_duplicate_positions(best_positions)

    # Fase 2: Otimização das Direções para a última posição dos drones
    print('---------------------------Descobrir Direções------------------------------')
    best_directions, best_info = ga.run_direction_optimization(best_positions)
    print(best_directions)
    bottleneck = best_info['Capacidade bottleneck mínima']
    cap_media=best_info['Capacidade média [Kbps]']
    
    env.posicoes = best_positions.reshape((env.num_uavs, 2))
    env.direcoes_antena=best_directions
    # Renderizar a solução encontrada
    env.render()

    # Calcular e imprimir o tempo de execução do ciclo
    cycle_duration = time.time() - start_time
    print(f"Tempo de execução do ciclo: {cycle_duration:.2f} segundos")

    current_time += update_interval




SAC

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
import os
import matplotlib.pyplot as plt

# Rede neural para os críticos (Q-networks)
class Critic(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        return self.fc5(x)

# Rede neural para o ator (policy network)
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 512)
        self.mu_layer = nn.Linear(512, output_dim)
        self.log_std_layer = nn.Linear(512, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        mu = self.mu_layer(x)
        log_std = self.log_std_layer(x)
        log_std = torch.clamp(log_std, -20, 2)  # Para estabilidade numérica
        return mu, log_std

    def sample(self, state):
        mu, log_std = self.forward(state)
        std = log_std.exp()
        normal = torch.distributions.Normal(mu, std)
        z = normal.rsample()  # Reparametrização
        action = torch.tanh(z)
        log_prob = normal.log_prob(z) - torch.log(1 - action.pow(2) + 1e-6)
        return action, log_prob.sum(dim=-1)


class ReplayBuffer:
    def __init__(self, max_size=100000):
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def size(self):
        return len(self.buffer)


class SACAgent:
    def __init__(self, state_dim, action_dim, action_range):
        self.actor = Actor(state_dim, action_dim)
        self.critic1 = Critic(state_dim + action_dim, 1)
        self.critic2 = Critic(state_dim + action_dim, 1)
        self.target_critic1 = Critic(state_dim + action_dim, 1)
        self.target_critic2 = Critic(state_dim + action_dim, 1)
        
        # Verifique as dimensões dos estados e ações
        print(f"State dim: {state_dim}, Action dim: {action_dim}")
        
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=1e-4)
        self.critic1_optimizer = optim.Adam(self.critic1.parameters(), lr=1e-4)
        self.critic2_optimizer = optim.Adam(self.critic2.parameters(), lr=1e-4)

        self.target_critic1.load_state_dict(self.critic1.state_dict())
        self.target_critic2.load_state_dict(self.critic2.state_dict())

        self.replay_buffer = ReplayBuffer()
        self.gamma = 0.99
        self.tau = 0.01
        self.alpha = 0.001  # Entropy coefficient
        self.action_range = action_range
        
        self.reward_history = []
        self.min_reward = float('inf')
        self.max_reward = -float('inf')

    def update_reward_statistics(self, reward):
        self.reward_history.append(reward)
        if reward < self.min_reward:
            self.min_reward = reward
        if reward > self.max_reward:
            self.max_reward = reward

    def normalize_reward(self, reward):
        self.update_reward_statistics(reward)
        if self.max_reward == self.min_reward:
            return 0.0  # Evita divisão por zero
        return (reward - self.min_reward) / (self.max_reward - self.min_reward)

    def save(self, filepath):
        # Salvar os modelos dos atores e críticos
        torch.save({
            'actor_state_dict': self.actor.state_dict(),
            'critic1_state_dict': self.critic1.state_dict(),
            'critic2_state_dict': self.critic2.state_dict(),
            'actor_optimizer_state_dict': self.actor_optimizer.state_dict(),
            'critic1_optimizer_state_dict': self.critic1_optimizer.state_dict(),
            'critic2_optimizer_state_dict': self.critic2_optimizer.state_dict(),
        }, filepath)
        print(f"Model saved to {filepath}")

    def load(self, filepath):
        # Carregar os modelos dos atores e críticos
        checkpoint = torch.load(filepath)
        self.actor.load_state_dict(checkpoint['actor_state_dict'])
        self.critic1.load_state_dict(checkpoint['critic1_state_dict'])
        self.critic2.load_state_dict(checkpoint['critic2_state_dict'])
        self.actor_optimizer.load_state_dict(checkpoint['actor_optimizer_state_dict'])
        self.critic1_optimizer.load_state_dict(checkpoint['critic1_optimizer_state_dict'])
        self.critic2_optimizer.load_state_dict(checkpoint['critic2_optimizer_state_dict'])
        print(f"Model loaded from {filepath}")
    
    
    def select_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0)  # Certifique-se de que o estado é um vetor e converta para tensor
        action, _ = self.actor.sample(state)
        return action.detach().numpy().flatten()

    def update(self, batch_size):
        if self.replay_buffer.size() < batch_size:
            return

        samples = self.replay_buffer.sample(batch_size)
        states, actions, rewards, next_states, dones = zip(*samples)
        states = torch.FloatTensor(np.array(states))
        actions = torch.FloatTensor(np.array(actions))
        rewards = torch.FloatTensor(rewards).unsqueeze(1)
        next_states = torch.FloatTensor(np.array(next_states))
        dones = torch.FloatTensor(dones).unsqueeze(1)

        with torch.no_grad():
            next_actions, next_log_probs = self.actor.sample(next_states)
            target_q1 = self.target_critic1(torch.cat([next_states, next_actions], 1))
            target_q2 = self.target_critic2(torch.cat([next_states, next_actions], 1))
            target_q = rewards + (1 - dones) * self.gamma * (torch.min(target_q1, target_q2) - self.alpha * next_log_probs.unsqueeze(1))

        q1 = self.critic1(torch.cat([states, actions], 1))
        q2 = self.critic2(torch.cat([states, actions], 1))
        critic1_loss = nn.MSELoss()(q1, target_q)
        critic2_loss = nn.MSELoss()(q2, target_q)

        self.critic1_optimizer.zero_grad()
        critic1_loss.backward()
        self.critic1_optimizer.step()

        self.critic2_optimizer.zero_grad()
        critic2_loss.backward()
        self.critic2_optimizer.step()

        actions, log_probs = self.actor.sample(states)
        q1_pi = self.critic1(torch.cat([states, actions], 1))
        q2_pi = self.critic2(torch.cat([states, actions], 1))
        actor_loss = (self.alpha * log_probs - torch.min(q1_pi, q2_pi)).mean()

        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Update target networks
        for target_param, param in zip(self.target_critic1.parameters(), self.critic1.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for target_param, param in zip(self.target_critic2.parameters(), self.critic2.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


# Inicialização do ambiente e do agente
env = UAVCommunicationEnv()  # Substitua pelo seu ambiente
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_range = [env.action_space.low, env.action_space.high]

# Inicialize o agente
agent = SACAgent(state_dim, action_dim, action_range)

# Certifique-se de que o diretório para salvar o modelo exista
os.makedirs("C:/Users/tass/OneDrive - EXÉRCITO PORTUGUÊS/Ambiente de Trabalho/Tese/Simulaçãoes/best_models", exist_ok=True)

# Inicialize a variável para monitoramento do melhor modelo
best_reward = -float('inf')  # Inicia com um valor muito baixo para comparação
reward_history = []
reward_history_not_normalized = []  # Lista para armazenar as recompensas de cada episódio
average_rewards = []  # Lista para armazenar a recompensa média a cada 50 episódios

recompensa = []  # Inicialize como uma lista
average_rewards_normalized = []

# Loop de treinamento
num_episodes = 2000
batch_size = 2048

for episode in range(num_episodes):
    state, info = env.reset()  # Desempacote a tupla retornada
    state = np.array(state).flatten()  # Converta o estado para um vetor, se necessário
    print(f"Initial state: {state}, Shape: {state.shape}")  # Debugging

    episode_reward = 0
    episode_reward_not_normalized = 0
    done = False

    action = agent.select_action(state)
    next_state, reward, done, info = env.step(action)
    next_state = np.array(next_state).flatten()  # Certifique-se de que o próximo estado é um vetor
    normalized_reward = agent.normalize_reward(reward)
    agent.replay_buffer.add((state, action, normalized_reward, next_state, done))
    agent.update(batch_size)
    state = next_state
    episode_reward += normalized_reward
    episode_reward_not_normalized += reward

    recompensa.append(reward)  # Adiciona a recompensa à lista

    reward_history.append(episode_reward)  # Adiciona a recompensa do episódio à lista
    reward_history_not_normalized.append(episode_reward_not_normalized)

    print(f"Episode: {episode}, Reward: {episode_reward}")

    # Salvar o melhor modelo
    if episode_reward > best_reward:
        best_reward = episode_reward
        agent.save("C:/Users/tass/OneDrive - EXÉRCITO PORTUGUÊS/Ambiente de Trabalho/Tese/Simulaçãoes/best_models/sac_best_model.pth")
        print(f"New best model saved with reward: {best_reward}")

    # Calcular e armazenar a recompensa média a cada 50 episódios
    if (episode + 1) % 50 == 0:
        avg_reward = np.mean(reward_history[-50:])
        average_rewards.append(avg_reward)
        print(f"Episode {episode + 1}: Average Reward over last 50 episodes: {avg_reward}")

# Normalização min-max das recompensas
min_recompensa = np.min(recompensa)
max_recompensa = np.max(recompensa)
Normalizada = (np.array(recompensa) - min_recompensa) / (max_recompensa - min_recompensa)

# Cálculo da média das recompensas normalizadas a cada 50 episódios
average_rewards_normalized = [np.mean(Normalizada[i:i+50]) for i in range(0, len(Normalizada), 50)]

# Plot das recompensas não normalizadas por episódio
# plt.figure(figsize=(10, 5))
# plt.plot(range(num_episodes), reward_history_not_normalized)
# plt.xlabel('Episódios')
# plt.ylabel('Recompensa Total')
# plt.title('Recompensa por Episódio (Não Normalizada)')
# plt.show()



agent = SACAgent(state_dim, action_dim, action_range)
agent.load("C:/Users/tass/OneDrive - EXÉRCITO PORTUGUÊS/Ambiente de Trabalho/Tese/Simulaçãoes/best_models/sac_best_model.pth")
env=UAVCommunicationEnv()
env.reset()
# Inicializar variáveis para armazenar a melhor recompensa e o melhor episódio
best_reward = -float('inf')
best_episode = None
best_episode_steps = []

# Executar o modelo no ambiente 100 vezes
num_test_episodes = 1000

for episode in range(num_test_episodes):
    state, _ = env.reset()
    episode_reward = 0
    episode_steps = []

    
    action = agent.select_action(state)  # Seleciona ação sem exploração adicional
    next_state, reward, done, info = env.step(action)
    episode_steps.append((state, action))  # Armazena o estado e a ação
    state = next_state
    episode_reward += reward

    # print(f"Test Episode: {episode}, Reward: {episode_reward}")

    # Se este episódio for o melhor até agora, salvá-lo
    if episode_reward > best_reward:
        best_reward = episode_reward
        best_episode = episode
        best_episode_steps = episode_steps

# Renderizar o melhor episódio
print(f"Best Episode: {best_episode}, Best Reward: {best_reward}")

# Resetar o ambiente
state, _ = env.reset()
# env.render()

for step in best_episode_steps:
    state, action = step
    env.step(action)
    obs, reward, done, info = env.step(action)
    title = 'Direções e Posições Estabelecidas - Algoritmo SAC'
    bottleneck = info['Capacidade bottleneck mínima']
    cap_media = info['Capacidade média [Kbps]']
    print(info['Capacidades_por_link [Kbps]'])
    env.render()

print("Renderização do melhor episódio concluída.")

